#unisabana #maestriaanalitica

#Cargue de tabla de anlítica a base de datos en AWS
Se inicia el proceso con el llamado a las librerías.

In [ ]:
import pandas as pd
import sqlalchemy
import psycopg2 as ps

Contexto: se realizó la extracion de datos de la base de datos original, a partir de consultas se construyó este set de datos que se ha cargado en una tabla, se ha puesto en un csv y ahora será cargada a la nube AWS.
**Objetivo.** realizar el carge de los datos limpios extraidos en una base de datos  configurada en el servicio de computación en la nube (AWS).

Con esta base de datos se generarán proceso de visualización.

# Conexión a la base de datos NBA creada en aws

Para la conexión a la base de datos, utilizamos los siguientes parámetros

- host : nba.cktm0x5r352o.us-east-1.rds.amazonaws.com
- puerto: 5434
- base de datos : postgres
- password : 12345678
- user: Postgres

Establecemos la conexión a la base de datos con SQLAlchemy, usaremos para ello los parametros de conexión ya mencionados, luego procederemos al cargue de la tabla de analitica a la base de datos postgres en aws.

In [ ]:
#parámetros de conexión AWS, variable de entorno
host = "nba.cktm0x5r352o.us-east-1.rds.amazonaws.com"
port = '5434'
user = 'postgres'
password = '12345678'
database = 'postgres' #verificar la existencia de la base datos

In [ ]:
#cargue de librería
from sqlalchemy import create_engine
#fucion de conexion a base de datos
connAlchemy = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = connAlchemy.raw_connection()
cur = conn.cursor()


### Analisis codigo
Conexión a la Base de Datos: Se establece una conexión a una base de datos PostgreSQL usando SQLAlchemy. Esta base de datos está alojada en AWS.

En este caso tambien se uso dbevear con el proposito de verificar la existencia de la base de datos y comprobar los datos de conexión.

### cargue de tabla para analitica y visualización
Usando la librería SQLAlchemy y mediante un proceso de ORM, cargamos la información de la tabla.

In [ ]:
#testing de conexión
try:
    connAWS = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}', echo=True , future=True)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

 Importamos módulos y clases de SQLAlchemy necesarios para realizar las tareas necesarias de manipulación y cargue de datos.  estas importaciones forman la base para configurar y manipular una base de datos utilizando SQLAlchemy de manera declarativa.

In [ ]:
from sqlalchemy import Column
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy import Float
from sqlalchemy import VARCHAR
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session

defición de la clase base para sqlachemy

In [ ]:
Base = declarative_base()

Dedinición de columnas y tipo de dato de las mismas.

In [ ]:
## Se genera la estrucutra de la tabla analítica definiendo las columnas y el tipo de dato
class Analitica(Base):
    __tablename__= "analitica"
    team_id = Column(Integer, primary_key=True)
    nickname = Column(VARCHAR(50))
    equipo = Column(VARCHAR(50))
    temporada = Column(Integer)
    wins = Column(Integer)
    puntos_marcados = Column(Float)
    puntos_contra = Column(Float)
    puntos_promedio = Column(Float)
    total_blocks = Column(Integer)
    avg_height_wo_shoes = Column(Float)
    avg_weight = Column(Float)
    avg_wingspan = Column(Float)
    avg_standing_reach = Column(Float)
    avg_max_vertical_leap = Column(Float)


def __repr__(self):
    return f"Analitica(team_id={self.team_id!r}, nickname={self.nickname!r}, equipo={self.equipo!r}, \
            temporada={self.temporada!r}, wins={self.wins!r}, puntos_marcados={self.puntos_marcados!r}, \
            puntos_contra={self.puntos_contra!r}, puntos_promedio={self.puntos_promedio!r}, total_blocks={self.total_blocks!r}, \
            avg_height_wo_shoess={self.avg_height_wo_shoes!r}, avg_weight={self.avg_weight!r}, avg_wingspan={self.avg_wingspan!r}, \
            avg_standing_reach={self.avg_standing_reach!r}, avg_max_vertical_leap={self.avg_max_vertical_leap!r}"

Creación del schema de la tabla en la base de datos en AWS usando el motor de conexión.

In [ ]:
Base.metadata.create_all(connAWS)

Lectura del contenido de la tabla y cargue en un dataframe.

In [ ]:
from sqlalchemy.orm import Session
analitica = pd.read_csv('C:/pr_bigdata/analitica_limpia_202405300005.csv')
anlitica = analitica.reset_index()
analitica.columns

Se carga el contenido de la tabla a la base de datos aws usando iteraciones.

In [ ]:

#Carga de las filas en la tabla analitica
with Session(connAWS) as session:
    for index, row in analitica.iterrows():
        job = Analitica(
                    nickname = row['nickname'],
                    equipo = row['equipo'],
                    temporada = row['temporada'],
                    wins = row['wins'],
                    puntos_marcados = row['puntos_marcados'],
                    puntos_contra = row['puntos_contra'],
                    puntos_promedio = row['puntos_promedio'],
                    total_blocks = row['total_blocks'],
                    avg_height_wo_shoes = row['avg_height_wo_shoes'],
                    avg_weight = row['avg_weight'],
                    avg_wingspan = row['avg_wingspan'],
                    avg_standing_reach = row['avg_standing_reach'],
                    avg_max_vertical_leap = row['avg_max_vertical_leap']
                    )

        session.add(job)
    session.commit()
